In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import sys
import os
import re

class Viewer(object):
    def __init__(self, path, prefix=None, magfile=None):
        if prefix is None: prefix = "zernikes"
        if magfile is None: magfile = "MAG.TXT"
        self.files = [{'filename':fn,
                       'offset': float(re.match(r".*([\+|-][0-9]+\.[0-9]+).*", fn).groups()[0]),}
                      for fn in os.listdir(path) if re.match(r"zernike", fn)]
        for f in self.files:
            with open(os.path.join(path, f['filename']), 'r', encoding='utf-16-le') as fh:
                # Skip header
                row = fh.readline().strip("\ufeff")
                while(row and not re.match(r"\s*Field:", row)):
                    row = fh.readline()
                raw = list(fh)
                data = []
                for row in raw:
                    data.append([float(col) for col in re.split(r"\s*", row.strip())])
                f['field'] = np.array([d[0] for d in data])
                f['zernike'] = np.array([d[1:] for d in data])
                
        with open(os.path.join(path, magfile), encoding='utf-16-le') as fh:
            row = fh.readline().strip("\ufeff")
            if re.match(r"Universal Plot", row):
                while(row and not re.match("\s*X\s*Y", row)):
                    row = fh.readline().strip("\ufeff")
                raw = list(fh)
                self.mag = np.split(np.array([[float(col) for col in re.split(r"\s*", row.strip())] 
                                                for row in raw]).T, 2)[1].flatten()
            else:
                fh.seek(0)
                raw = list(fh)
                self.mag = np.array([float(row.strip('\ufeff')) for row in raw])

        self.field = self.files[0]['field']
        order = np.argsort([f['offset'] for f in self.files])
        self.offset = np.array([self.files[o]['offset'] for o in order])
        self.alldata = np.array([self.files[o]['zernike'] for o in order])
        

    def surface(self, mode):
        # Surface plot for one mode.
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(self.offset, self.field)
        ax.plot_surface(X,Y, self.alldata[:,:,mode].T, rstride=1, cstride=1)
        fig.show()
        

    def do_interp(self, offset):
        data = np.zeros_like(self.files[0]['zernike'])
        for z in range(data.shape[1]):
            for f in range(data.shape[0]):
                data[f,z] = np.interp([offset], self.offset, self.alldata[:,f,z])
            mag = np.interp(offset, self.offset, self.mag)
        return data, mag

        
    def plot(self):
        # Interactive plot for all modes.
        fig, ax = plt.subplots()
        plt.subplots_adjust(left=0.1, bottom=0.25, right=0.8)
        axoffset = plt.axes([0.1, 0.1, 0.7, 0.03])

        data, mag0 = self.do_interp(0)
        lines = ax.plot(self.field, data)
        for (i, l) in enumerate(lines):
            l.set_linestyle(['-','-.',':','--'][int(i/7)])

        ax.set_xlim(self.field.min(), self.field.max())
        ax.set_ylim(self.alldata.min(), self.alldata.max())
        ax.set_xlabel('field / mm')
        ax.set_ylabel('Zernike coefficient / waves')
        ax.legend([l for l in lines], range(len(lines)), loc='center left', 
          bbox_to_anchor=(1.05, 0.5), 
          bbox_transform=ax.transAxes, 
          title="Standard\nZernike\ncoefficient")

        soffset = Slider(axoffset, '', self.offset.min(), self.offset.max(), 
                         valinit=0, closedmin=True, closedmax=True)
        axoffset.set_xlabel('offset / mm')

        def update(val):
            offset = soffset.val
            soffset.poly.set_color(['red','blue'][offset >= soffset.valinit])
            data, mag = self.do_interp(offset)
            ax.set_title("{:.3f}%".format(float(100*(mag-mag0)/mag0)))
            for i, l in enumerate(lines):
                l.set_ydata(data[:,i])
        
        update(0)
        soffset.on_changed(update)

        plt.show()
        
        
v0 = Viewer(os.path.join(".", "4PiZernikes"), "zernikes", "MAG.TXT")
v1 = Viewer(os.path.join(".", "4PiZernikes", "MoveFirstLens"), "zernikes", "AMAG.TXT")
v2 = Viewer(os.path.join(".", "4PiZernikes", "MoveSecondLens"), "zernikes", "AMAG.TXT")

v0.plot()
v1.plot()
v2.plot()

Using matplotlib backend: TkAgg


C:\python3\python-3.5.2.amd64\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
